In [1]:
# Import basic libraries
from palmerpenguins import load_penguins
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import time
from sklearn.preprocessing import LabelEncoder
import os
os.chdir("/Users/hoangthuyduongvu/Desktop/FuzzSIM")


In [2]:
# Import personalized libraries
from fuzz.src.capacity import *
from fuzz.choquet.choquet import *
from fuzz.src.norm import *
from fuzz.src.knn import KNNFuzz
from fuzz.src.sim import S1, S2, S3
from fuzz.optim import *
from fuzz.utils import *
from fuzz.eval import leave_one_out
from fuzz.dataloader import *
from fuzz.choquet.d_choquet import *

In [43]:
def generate_mobius(feature_indices: List[int]) -> List[Capacity]:
    """Generate a random 2-additive Möbius measure."""
    m = {}
    # Singleton terms
    for i in feature_indices:
        m[frozenset([i])] = np.random.rand()
    
    # Pairwise interaction terms (2-additive)
    for i, j in combinations(feature_indices, 2):
        m[frozenset([i, j])] = np.random.rand()

    m[frozenset()] = 0.0

    tmp = []
    # Convert to fit with Capacity implementation
    for k, v in m.items():
        tmp.append(Capacity(list(k), v))
    return tmp

def mobius_to_capacity(m: List[Capacity], feature_indices: List[int]) -> List[Capacity]:
    """
    Convert Möbius transform to capacity.
    m: mobius 
    """
    mu = []
    for subset in powerset(feature_indices):
        fs_subset = frozenset(subset)
        total = 0.0
        for B in powerset(subset):
            fs_B = list(frozenset(B))
            tmp = locate_capacity(X=fs_B, capacity=m)
            # print(f"fs_B: {fs_B}, tmp: {tmp}")
            # if fs_B in m:
            total += tmp
        mu.append(Capacity(list(fs_subset), total))

    def norm_capacity(capacity: List[Capacity]) -> List[Capacity]:
        """Normalize the capacity."""
        lst = [c.mu for c in capacity if c.mu is not None]
        min_lst = min(lst)
        max_lst = max(lst)
        # Normalize to [0, 1]
        lst = norm(lst)
        tmp = []
        for i in range(len(lst)):
            tmp.append(Capacity(capacity[i].X, lst[i]))
        return tmp
    return norm_capacity(mu)

In [44]:
features = [0, 1, 2, 3]

mobius = generate_mobius(features)
capacity = mobius_to_capacity(mobius, features)

print("Möbius:")
for i in range(len(mobius)):
    print(f"Mobius of {mobius[i].X} is {mobius[i].mu:.3f}")

print("\nDerived Capacity:")
for c in capacity:
    print(f"Capacity of {c.X} is {c.mu:.3f}")


Möbius:
Mobius of [0] is 0.878
Mobius of [1] is 0.899
Mobius of [2] is 0.624
Mobius of [3] is 0.999
Mobius of [0, 1] is 0.938
Mobius of [0, 2] is 0.875
Mobius of [0, 3] is 0.444
Mobius of [1, 2] is 0.316
Mobius of [1, 3] is 0.974
Mobius of [2, 3] is 0.682
Mobius of [] is 0.000

Derived Capacity:
Capacity of [] is 0.000
Capacity of [0] is 0.115
Capacity of [1] is 0.118
Capacity of [2] is 0.082
Capacity of [3] is 0.131
Capacity of [0, 1] is 0.356
Capacity of [0, 2] is 0.312
Capacity of [0, 3] is 0.304
Capacity of [1, 2] is 0.241
Capacity of [1, 3] is 0.377
Capacity of [2, 3] is 0.302
Capacity of [0, 1, 2] is 0.594
Capacity of [0, 1, 3] is 0.673
Capacity of [0, 2, 3] is 0.590
Capacity of [1, 2, 3] is 0.589
Capacity of [0, 1, 2, 3] is 1.000


In [18]:
for subset in powerset(features):
    print(f"Subset: {set(subset)}")

Subset: set()
Subset: {0}
Subset: {1}
Subset: {2}
Subset: {3}
Subset: {0, 1}
Subset: {0, 2}
Subset: {0, 3}
Subset: {1, 2}
Subset: {1, 3}
Subset: {2, 3}
Subset: {0, 1, 2}
Subset: {0, 1, 3}
Subset: {0, 2, 3}
Subset: {1, 2, 3}
Subset: {0, 1, 2, 3}


In [19]:
X = np.array([0.2, 0.5, 0.8])
choquet_value = Choquet_classic(X, capacity, verbose=True)
print("Choquet integral:", choquet_value)


val_check: [0, 1, 2] - capacity_observation_i: 3.0477897080985366 - val_check2: [1, 2] - capacity_observation_i_1: 1.1313811569511114
val_check: [1, 2] - capacity_observation_i: 1.1313811569511114 - val_check2: [2] - capacity_observation_i_1: 0.47940945914150856
val_check: [2] - capacity_observation_i: 0.47940945914150856 - val_check2: [] - capacity_observation_i_1: 0.0
Choquet integral: 1.0927951264474933


In [20]:
iris_data = load_iris()
iris = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
iris['target'] = iris_data.target
data, labels = iris.iloc[:, :-1].values, iris.iloc[:, -1].values
data = batch_norm(np.array(data, dtype=np.float32))

In [23]:
choquet = Choquet(X=data[0], mu=capacity, version='classic', p=1, q=1)
print(choquet.choquet)
# choquet.Choquet_classic(verbose=True)

3.7917923366899426


In [24]:
X = data[0]
choquet_value = Choquet_classic(X, capacity, verbose=True)
print("Choquet integral:", choquet_value)


val_check: [3, 2, 1, 0] - capacity_observation_i: 5.635291663509719 - val_check2: [2, 1, 0] - capacity_observation_i_1: 3.0477897080985366
val_check: [2, 1, 0] - capacity_observation_i: 3.0477897080985366 - val_check2: [1, 0] - capacity_observation_i_1: 1.64441463677246
val_check: [1, 0] - capacity_observation_i: 1.64441463677246 - val_check2: [0] - capacity_observation_i_1: 0.58617683971143
val_check: [0] - capacity_observation_i: 0.58617683971143 - val_check2: [] - capacity_observation_i_1: 0.0
Choquet integral: 3.7917923366899426


In [25]:
# Each individual is a dictionary of Möbius values
def mutate(mobius, mutation_rate=0.1):
    new_mobius = mobius.copy()
    for i in range(len(mobius)):
        if np.random.rand() < mutation_rate:
            new_mobius[i] = Capacity(mobius[i].X, np.clip(mobius[i].mu + np.random.uniform(-0.1, 0.1), 0, 1))
    return new_mobius

In [28]:
def crossover(parent1, parent2):
    """Suppose that 2 parents have the same structure with different capacities."""
    child = []
    # Get len parents 
    if len(parent1) != len(parent2):
        raise ValueError("Parents must have the same length")
    len_parents = len(parent1)
    for i in range(len_parents):
        c_1 = parent1[i].mu
        c_2 = parent2[i].mu
        c_c = random.choice([c_1, c_2])
        if c_c == c_1:
            child.append(Capacity(parent1[i].X, c_1))
        else:
            child.append(Capacity(parent2[i].X, c_2))
    return child

In [36]:
cross = crossover(
    parent1=generate_capacity(enumerate_permute_unit(data[0])), 
    parent2=generate_capacity(enumerate_permute_unit(data[0]))
)
print("\nCrossed Capacity:")
for c in cross:
    print(f"Capacity of {c.X} is {c.mu:.3f}")


Crossed Capacity:
Capacity of [] is 0.000
Capacity of [3] is 0.024
Capacity of [2] is 0.130
Capacity of [1] is 0.145
Capacity of [0] is 0.167
Capacity of [3, 2] is 0.355
Capacity of [3, 1] is 0.501
Capacity of [3, 0] is 0.519
Capacity of [2, 1] is 0.568
Capacity of [2, 0] is 0.735
Capacity of [1, 0] is 0.832
Capacity of [3, 2, 1] is 0.834
Capacity of [3, 2, 0] is 0.872
Capacity of [3, 1, 0] is 0.919
Capacity of [2, 1, 0] is 0.948
Capacity of [3, 2, 1, 0] is 1.000


In [27]:
mutated_mobius = mutate(mobius, mutation_rate=0.2)
print("\nMutated Möbius:")
for i in range(len(mutated_mobius)):
    print(f"Mobius of {mutated_mobius[i].X} is {mutated_mobius[i].mu:.3f}")


Mutated Möbius:
Mobius of [0] is 0.573
Mobius of [1] is 0.169
Mobius of [2] is 0.479
Mobius of [3] is 0.974
Mobius of [0, 1] is 0.926
Mobius of [0, 2] is 0.404
Mobius of [0, 3] is 0.639
Mobius of [1, 2] is 0.520
Mobius of [1, 3] is 0.279
Mobius of [2, 3] is 0.695
Mobius of [] is 0.000
